In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
pwd

'c:\\Users\\el papi\\smart-trash-classifier'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

In [ ]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import torch
import time

In [7]:
import os
import time
import torch
from torch.utils.tensorboard import SummaryWriter



class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config
        self.writer = None
        self.ckpt_path = self.config.checkpoint_model_filepath

    def _create_tb_writer(self):
        """Create a TensorBoard writer with timestamped log dir"""
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        os.makedirs(tb_running_log_dir, exist_ok=True)
        self.writer = SummaryWriter(log_dir=tb_running_log_dir)
        print(f"[TensorBoard] Logging at: {tb_running_log_dir}")
        return self.writer

    def _save_checkpoint(self, model, optimizer, epoch, val_loss):
        """Save model checkpoint if it's the best so far"""
        os.makedirs(os.path.dirname(self.ckpt_path), exist_ok=True)

        checkpoint = {
            "epoch": epoch,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "val_loss": val_loss,
        }
        torch.save(checkpoint, self.ckpt_path)
        print(f"[Checkpoint] Model saved at {self.ckpt_path}")

    def get_tb_writer(self):
        """Public method to get TensorBoard writer"""
        if self.writer is None:
            self._create_tb_writer()
        return self.writer


In [11]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_writer()
    
except Exception as e:
    raise e

[2025-09-27 21:01:02,477: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-27 21:01:02,481: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-27 21:01:02,484: INFO: common: created directory at: artifacts]
[2025-09-27 21:01:02,486: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2025-09-27 21:01:02,488: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[TensorBoard] Logging at: artifacts\prepare_callbacks\tensorboard_log_dir\tb_logs_at_2025-09-27-21-01-02
